# Template Jupyter Notebook

In [15]:
import json
import codecs
import unicodedata
import string
from collections import Counter
import re
import pandas as pd

In [16]:
def parse_json(path: str):
    with open(path, "r", encoding="utf-8") as file:
        raw_data = json.load(file)
    
    messages = pd.DataFrame(raw_data["messages"])
    participants = pd.DataFrame(raw_data["participants"])
    
    messages["content"] = messages["content"].apply(clean_text)
    messages["sender_name"] = messages["sender_name"].apply(clean_text)
    participants["name"] = participants["name"].apply(clean_text)
    
    return messages, participants 

In [17]:
def get_conversation(messages: pd.DataFrame, totalConversation: dict):
    for index, row in messages.iterrows():
        sender = row.get("sender_name")
        content = row.get("content")
        if content == None:
            continue
        totalConversation.setdefault(sender, []).append(content)
    return totalConversation

In [18]:
def clean_text(text):
    if not isinstance(text, str):
        return text
    try:
        text = text.encode("latin1").decode("utf-8")
    except Exception:
        pass
    text = text.replace("\u200d", "")   # remove zero-width joiner
    return text

In [19]:
def word_frequency(conversations: dict, targetWord: str):
    targetWord = targetWord.lower().strip()
    word_count = {}

    for sender, messages in conversations.items():  # ← FIXED
        count = 0
        for msg in messages:
            # Skip invalid values
            if not isinstance(msg, str) or not msg.strip():
                continue

            lower_msg = msg.lower()

            # Count if target word appears
            if targetWord in lower_msg:
                count += 1

        # Always include user even if count = 0
        word_count[sender] = count

    return word_count

In [20]:
def top_word_per_user(conversation: dict, user: str, top_n: int = 10):
    # Check if user exists
    if user not in conversation:
        print(f"User '{user}' not found.")
        return {}

    messages = conversation[user]
    words = []

    for msg in messages:
        if not isinstance(msg, str) or not msg.strip():
            continue

        lower_msg = msg.lower()

        for w in lower_msg.split():
            clean_word = w.strip(string.punctuation)
            if clean_word:
                words.append(clean_word)

    counter = Counter(words)
    return dict(counter.most_common(top_n))

In [21]:
messages, participants = parse_json("message_1.json")
result = get_conversation(messages, {})
word_freq = word_frequency(result, "fuck")
word_per = top_word_per_user(result,"Austen Brown")
print(word_per)

NameError: name 'Counter' is not defined